In [1]:
#Import Libraries
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship

#### Neo4j Sandbox Connection 

In [26]:
uri = 'bolt://35.175.116.172:7687'
pwd = 'alternation-pyramid-semiconductor'
user = 'neo4j'
# give your url to .csv file
dataUrl = "'https://gist.githubusercontent.com/yusufakcakaya/2ca1f7387bbcb7515b429c29c50bd4c8/raw/684d51f4eccbd518b3d4f073667d20ed4b29cf9b/dataset'"


In [18]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [19]:
#Object of Neo4jConnection 
conn = Neo4jConnection(uri=uri, user= user, pwd=pwd)


In [42]:
#Before creating labels delete all

conn.query('MATCH (n) DETACH DELETE n')

[]

In [43]:
#Source entities
conn.query('LOAD CSV WITH HEADERS FROM '+ dataUrl + 
' AS line CREATE (source:SOURCE {name:line.source})')

#Target entities
conn.query('LOAD CSV WITH HEADERS FROM '+ dataUrl + 
' AS line CREATE (target:TARGET {name:line.target})')


[]

In [44]:
# Create Relations
conn.query('LOAD CSV WITH HEADERS FROM '+ dataUrl + 
''' AS line MATCH (source:SOURCE {name:line.source})
MATCH (target:TARGET {name:line.target})
CREATE (source)-[:RELATED_TO{type:line.edge}]->(target)''')


[]

In [45]:
conn.query('MATCH (n) RETURN n')


[<Record n=<Node id=45 labels=frozenset({'SOURCE'}) properties={'name': 'June year  forecast'}>>,
 <Record n=<Node id=46 labels=frozenset({'SOURCE'}) properties={'name': 'two Paul Keating'}>>,
 <Record n=<Node id=47 labels=frozenset({'SOURCE'}) properties={'name': 'annual  Keating'}>>,
 <Record n=<Node id=48 labels=frozenset({'SOURCE'}) properties={'name': 'kind World War Keating'}>>,
 <Record n=<Node id=49 labels=frozenset({'SOURCE'}) properties={'name': 'Australia'}>>,
 <Record n=<Node id=50 labels=frozenset({'TARGET'}) properties={'name': '87  budget'}>>,
 <Record n=<Node id=51 labels=frozenset({'TARGET'}) properties={'name': 'two  Parliament'}>>,
 <Record n=<Node id=52 labels=frozenset({'TARGET'}) properties={'name': 'figures'}>>,
 <Record n=<Node id=53 labels=frozenset({'TARGET'}) properties={'name': 'Australia trading holocaust'}>>,
 <Record n=<Node id=54 labels=frozenset({'TARGET'}) properties={'name': 'recession'}>>]

In [46]:
allRelations = conn.query('MATCH ()-->() RETURN count(*);')     
allRelations


[<Record count(*)=5>]

In [47]:
 #List node labels
conn.query('CALL db.labels()')


[<Record label='SOURCE'>, <Record label='TARGET'>]

In [48]:
 #Relations
conn.query('CALL db.schema.visualization()')

[<Record nodes=[<Node id=-4 labels=frozenset({'TARGET'}) properties={'name': 'TARGET', 'indexes': [], 'constraints': []}>, <Node id=-3 labels=frozenset({'SOURCE'}) properties={'name': 'SOURCE', 'indexes': [], 'constraints': []}>] relationships=[<Relationship id=-2 nodes=(<Node id=-3 labels=frozenset({'SOURCE'}) properties={'name': 'SOURCE', 'indexes': [], 'constraints': []}>, <Node id=-4 labels=frozenset({'TARGET'}) properties={'name': 'TARGET', 'indexes': [], 'constraints': []}>) type='RELATED_TO' properties={}>]>]